Document for Model training



Importing packages

In [64]:
import random
import math
import matplotlib.pyplot as plt
from PIL import ImageDraw, Image
import bifurcationMathFunctions as bmf

Setting Constants

In [65]:
startX = 5
startY = 200
startAngle = 0
maxDepth = 15
lengthMedian = 20
lengthStdVar = 5
biforcProb = 0.3
bifurcBigLeft = 0.5
angleBiforc = math.pi/8
angleStdDev = math.pi/16

Auxillary Functions

In [71]:
def newAngle(angle):
    return random.gauss(angle, angleStdDev)

def newCoords(coord, angle, width):
    length = newLength(width)
    x,y = coord
    newX = x + math.cos(angle)*length
    newY = y + math.sin(angle)*length
    return newX,newY

def newLength(width):
    return 10*width + random.gauss(0, 2*width)

Synthetic Angiogram Class: Node

In [73]:
class Node:
    def __init__(self, coord, angle, width):
        self.coord = coord
        self.width = width
        self.prev_length = newLength(width)
        self.angle = angle
        self.children = []

    #Creates a child node
    def createChild(self,angle):
        newCoord = newCoords(self.coord, angle)
        return Node(newCoord, angle, self.width-1)

    #Creates and add either 1 or 2 children to the parent node.
    def addChildren(self):
        biforcation = (random.random() < biforcProb)
        if biforcation:
            #Make 2 children mirrored across an angle
            if bifurcBigLeft < random.random():
                angle1, angle2 = bmf.getThetas(0.5) #branches same thickness for now
            else:
                angle2, angle1 = bmf.getThetas(0.5)
            leftChild = self.createChild(self.angle - angle1)#(se1f.angle + angleBiforc)
            self.children.append(leftChild)
            rightChild = self.createChild(self.angle + angle2)#(self.angle - angleBiforc)
            self.children.append(rightChild)
        else:
            #Make 1 child following an angle
            singleChild = self.createChild(self.angle)
            self.children.append(singleChild)

Synthetic Angiogram Class: Tree

In [68]:
class Tree:
    def __init__(self, x, y, startAngle, width):
        self.addRoot(x,y, startAngle, width)
        self.makeTree()

    def addRoot(self, x, y, startAngle, width):
        self.root = Node((x,y), startAngle, width)
    
    def growTree(self, node):
        if node.width == 1:
            return
        else:
            node.addChildren()
            for child in node.children:
                self.growTree(child)
    
    def makeTree(self):
        self.growTree(self.root)
        

Drawing Tree

In [69]:
def nodeInside(cx, cy, X, Y):
    return (cx < X and cx >= 0 and cy < Y and cy >= 0)

def drawNode(node, draw):
    if len(node.children) >= 1:
        px, py = int(node.coord[0]), int(node.coord[1])
        for child in node.children:
            cx, cy = int(child.coord[0]), int(child.coord[1])
            draw.line((px, py, cx, cy), fill = 255, width=child.width)
            drawNode(child, draw)
    else:
        return

#Makes a treewalk, and finds minimum and maximum x, y values for tree
def drawTree(tree, dim):
    img = Image.new("1", dim)
    draw = ImageDraw.Draw(img)
    drawNode(tree.root, draw)
    img.show()
    return img
#help function that draws line from parent to each of its children. Should check for borders

Executing

In [70]:
tree = Tree(startX, startY, startAngle, maxDepth)
synAngio = drawTree(tree, (400, 400))

TypeError: newLength() missing 1 required positional argument: 'width'

In [54]:
#synAngio.save("test_image.PNG")